In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import pandas as pd

In [24]:
import geocoder

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
page = requests.get(url)

In [5]:
soup = BeautifulSoup(page.text, 'html.parser')

In [6]:
table = soup.find("table",{"class":"wikitable sortable"})

In [7]:
table_rows = table.find_all('tr')

In [8]:
table_rows[0].find_all('th')

[<th>Postcode</th>, <th>Borough</th>, <th>Neighbourhood
 </th>]

In [9]:
head = table_rows[0]
columns = head.text.split('\n')[1:4]

In [10]:
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    if row:
        l.append(row)

In [11]:
df = pd.DataFrame(l, columns=columns)

In [12]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [13]:
# remove the 'Not assigned' for Borough
df = df[df['Borough'] != "Not assigned"]

In [14]:
df['Neighbourhood']

2                     Parkwoods\n
3              Victoria Village\n
4                  Harbourfront\n
5              Lawrence Heights\n
6                Lawrence Manor\n
                  ...            
281    Kingsway Park South West\n
282                   Mimico NW\n
283          The Queensway West\n
284       Royal York South West\n
285              South of Bloor\n
Name: Neighbourhood, Length: 210, dtype: object

In [15]:
# remove '\n' from each string
df = df.replace('\n','', regex=True)

In [16]:
df['Neighbourhood']

2                     Parkwoods
3              Victoria Village
4                  Harbourfront
5              Lawrence Heights
6                Lawrence Manor
                 ...           
281    Kingsway Park South West
282                   Mimico NW
283          The Queensway West
284       Royal York South West
285              South of Bloor
Name: Neighbourhood, Length: 210, dtype: object

In [17]:
# combine 'Neighbourhood' for Postcode, Borough
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [18]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [19]:
# verify if Neighbourhood is 'Not assigned'
lo = df[df['Neighbourhood'] == 'Not assigned' ].index.values.astype(int)

# rename 'Neighbourhood' with 'Borough' 
for l in lo:
    df.loc[l]['Neighbourhood'] = df.loc[l]['Borough']

In [20]:
df[df['Neighbourhood'] == 'Not assigned' ]

,Postcode,Borough,Neighbourhood


In [21]:
df.shape

(103, 3)

In [30]:
geo_coord = pd.read_csv('Geospatial_Coordinates.csv')

In [31]:
geo_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [38]:
# changing 'Postal Code' to 'Postcode'
geo_coord.columns = ['Postcode', 'Latitude','Longitude']

In [39]:
geo_coord.columns

Index(['Postcode', 'Latitude', 'Longitude'], dtype='object')

In [43]:
# merge geo cordinates on dataframe
df = df.merge(geo_coord, on='Postcode', how='left')

In [44]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [46]:
# we can see that we have 2 additional columns
df.shape

(103, 5)